# 🌍 Stockholm Archipelago Trail POC
* see https://github.com/salgo60/Stockholm_Archipelago_Trail/issues/101
* [video](https://youtu.be/bepljHYFqp4) / [video 2](https://youtu.be/fBzhs2LQy_w)

In [42]:
import requests
import pandas as pd
import folium
import ipywidgets as widgets
from IPython.display import display, clear_output

# SPARQL endpoint
endpoint_url = "https://query.wikidata.org/sparql"

# Language groups
swedish_official = {'sv', 'fi', 'se', 'me', 'ri', 'sm', 'fit', 'yi', 'rmy'}
nordic_languages = swedish_official.union({'da', 'no', 'nn', 'is', 'fo'})
european_languages = {
    'en', 'fr', 'de', 'es', 'it', 'pt', 'pl', 'nl', 'cs', 'hu', 'ro', 'bg', 'el', 'tr', 'et', 'lv', 'lt', 'sk', 'sl', 'hr', 'mt',
    'da', 'fi', 'sv', 'no', 'is'
}  

# Language display mapping
language_names = {
    'sv': 'Swedish', 'en': 'English', 'fr': 'French', 'de': 'German', 'nn': 'Norwegian Nynorsk', 'nb': 'Norwegian Bokmål',
    'it': 'Italian', 'pl': 'Polish', 'es': 'Spanish', 'pt': 'Portuguese', 'ar': 'Arabic', 'ru': 'Russian', 'zh': 'Chinese',
    'fi': 'Finnish', 'da': 'Danish', 'nl': 'Dutch', 'jp': 'Japanese', 'fa': 'Persian', 'uk': 'Ukrainian', 'ku': 'Kurdish',
    'fit': 'Meänkieli', 'se': 'Northern Sami', 'sma': 'Southern Sami', 'smj': 'Lule Sami', 'sje': 'Pite Sami', 'sju': 'Ume Sami',
    'rmy': 'Romani', 'yi': 'Yiddish', 'et': 'Estonian', 'lv': 'Latvian', 'lt': 'Lithuanian', 'cs': 'Czech', 'hu': 'Hungarian',
    'el': 'Greek', 'tr': 'Turkish', 'ko': 'Korean', 'hi': 'Hindi', 'th': 'Thai', 'vi': 'Vietnamese', 'he': 'Hebrew',
    'id': 'Indonesian', 'ms': 'Malay', 'is': 'Icelandic', 'fo': 'Faroese'
}

all_languages = list(language_names.keys())

# Autodetect browser/system language with fallback to Swedish
import locale

def detect_language():
    try:
        lang = locale.getlocale()[0]
    except:
        lang = None
    if lang and lang[:2] in all_languages:
        return lang[:2]
    return 'sv'

# Dropdown for language selection with full names
language_selector = widgets.Dropdown(
    options=sorted([(f"{language_names[code]} - {code}", code) for code in all_languages]),
    value=detect_language(),
    description='Language:',
    disabled=False
)


# Instance type list from user (simplified display, filtering using full list)
custom_instances = [
    ('All', ''),
    ('Systembolagets ombud', 'wd:Q134529187'),
    ('Apotek', 'wd:Q13107184'),
    ('Arbetslivsmuseum', 'wd:Q33506'),
    ('Badplats', 'wd:Q567998'),
    ('Bageri', 'wd:Q274393'),
    ('Bastu', 'wd:Q57036'),
    ('Begravningsplats', 'wd:Q39614'),
    ('Bensinstation', 'wd:Q205495'),
    ('Biograf', 'wd:Q41253'),
    ('Bondgård', 'wd:Q72030539'),
    ('Brygga', 'wd:Q133867301'),
    ('Butik', 'wd:Q213441'),
    ('By', 'wd:Q532'),
    ('Byggnad', 'wd:Q41176'),
    ('Campingplats', 'wd:Q27108230'),
    ('Cykeluthyrning', 'wd:Q134529179'),
    ('Dricksvatten', 'wd:Q7892'),
    ('Fyr', 'wd:Q39715'),
    ('Fågelstation', 'wd:Q1365207'),
    ('Glamping', 'wd:Q2153744'),
    ('Grav', 'wd:Q173387'),
    ('Grillplats', 'wd:Q1546788'),
    ('Grotta', 'wd:Q35509'),
    ('Gruva', 'wd:Q820477'),
    ('Gästhamn', 'wd:Q10512405'),
    ('Gård', 'wd:Q131596'),
    ('Gårdsbutik', 'wd:Q1371823'),
    ('Hamn', 'wd:Q44782'),
    ('Hamnkontor', 'wd:Q55076881'),
    ('Hembygdsgård', 'wd:Q10520688'),
    ('Hembygdsmuseum', 'wd:Q33506'),
    ('Hjärtstartare', 'wd:Q1450682'),
    ('Hotell', 'wd:Q27686'),
    ('Insjö', 'wd:Q23397'),
    ('Jordbruk', 'wd:Q11451'),
    ('Jungfrudans', 'wd:Q1937879'),
    ('Jättegryta', 'wd:Q1358604'),
    ('Kafé', 'wd:Q30022'),
    ('Kajakuthyrning', 'wd:Q134539211'),
    ('Kapell', 'wd:Q108325'),
    ('Kiosk', 'wd:Q693369'),
    ('Krog', 'wd:Q256020'),
    ('Kustartilleribatteri', 'wd:Q16536851'),
    ('Kyrka', 'wd:Q16970'),
    ('Kyrkogård', 'wd:Q39614'),
    ('Lanthandel', 'wd:Q1295201'),
    ('Livsmedelsbutik', 'wd:Q1295201'),
    ('Lotsstation', 'wd:Q16948701'),
    ('Minneslund', 'wd:Q39614'),
    ('Minnesmärke', 'wd:Q5003624'),
    ('Museijärnväg', 'wd:Q420962'),
    ('Museum', 'wd:Q33506'),
    ('Naturhamn', 'wd:Q283202'),
    ('Naturreservat', 'wd:Q179049'),
    ('Paviljong', 'wd:Q276173'),
    ('Pensionat', 'wd:Q1065252'),
    ('Pub', 'wd:Q212198'),
    ('Restaurang', 'wd:Q11707'),
    ('Rum och frukost', 'wd:Q27686'),
    ('Ryssugn', 'wd:Q10658341'),
    ('Samhälle', 'wd:Q486972'),
    ('Skola', 'wd:Q3914'),
    ('Skolenhet', 'wd:Q3914'),
    ('Småort i Sverige', 'wd:Q14839548'),
    ('Snorkelled', 'wd:Q134078772'),
    ('Staty', 'wd:Q179700'),
    ('Stugby', 'wd:Q1406318'),
    ('Stuguthyrning', 'wd:Q135107662'),
    ('Teaterkonst', 'wd:Q11635'),
    ('Telegraf', 'wd:Q6987428'),
    ('Turistattraktion', 'wd:Q570116'),
    ('Tältplats', 'wd:Q832778'),
    ('Tätort i Sverige', 'wd:Q12813115'),
    ('Udde', 'wd:Q191992'),
    ('Utegym', 'wd:Q692630'),
    ('Utsiktsplats', 'wd:Q6017969'),
    ('Vandrarhem', 'wd:Q654772'),
    ('Vandringsled', 'wd:Q2143825'),
    ('Vindskydd', 'wd:Q1797440'),
    ('Väderkvarn', 'wd:Q38720'),
    ('Vägvisare', 'wd:Q1937027')
]

instance_selector = widgets.Dropdown(
    options=custom_instances,
    value='',
    description='Instance:',
    disabled=False,
)

def fetch_osm_trail_geojson():
    overpass_url = "https://overpass-api.de/api/interpreter"
    query_wd = """
    SELECT ?relation WHERE {
      ?item wdt:P361 wd:Q131318799;
            wdt:P402 ?relation.
    }
"""

    headers = {"Accept": "application/sparql-results+json"}
    response = requests.get(endpoint_url, params={'query': query_wd}, headers=headers)
    response.raise_for_status()
    results = response.json()
    relation_ids = [int(binding['relation']['value']) for binding in results['results']['bindings']]
    query = (
    "[out:json];("
    + "".join([f"relation({rid});" for rid in relation_ids])
    + ");>;\nout geom;")    

    response = requests.get(overpass_url, params={'data': query})
    response.raise_for_status()
    data = response.json()

    geojson = {
        "type": "FeatureCollection",
        "features": []
    }

    for element in data['elements']:
        if element['type'] == 'way' and 'geometry' in element:
            coords = [(pt['lat'], pt['lon']) for pt in element['geometry']]
            feature = {
                "type": "Feature",
                "geometry": {
                    "type": "LineString",
                    "coordinates": [[lon, lat] for lat, lon in coords]
                },
                "properties": {
                    "name": element.get('tags', {}).get('name', 'SAT Trail Segment')
                }
            }
            geojson['features'].append(feature)
    return geojson  
    
def fetch_and_show_map(lang='en', instance=''):
    filter_clause = f"?id wdt:P31 {instance} ." if instance else ""

    query = f"""
    SELECT DISTINCT ?id ?geo ?hexcolor ?iconname
           (SAMPLE(?labelLang) AS ?labelLangVal)
           (SAMPLE(?descLang) AS ?descLangVal)
           (SAMPLE(?svLabel) AS ?svLabelVal)
           (SAMPLE(?svdesc) AS ?svdescVal)
           (SAMPLE(?svwiki) AS ?svwikiVal)
           (SAMPLE(?img) AS ?imgVal)
           (SAMPLE(?p3749) AS ?p3749Val)
           (SAMPLE(?website) AS ?websiteVal)
           (SAMPLE(?facebook) AS ?facebookVal)
           (SAMPLE(?instagram) AS ?instagramVal)
           (SAMPLE(?location) AS ?locationVal) 
           (SAMPLE(?extraImage) AS ?extraImageVal)
    WHERE {{
      ?id wdt:P6104 wd:Q134294510.
    
      OPTIONAL {{ ?id wdt:P625 ?geo . }}

      OPTIONAL {{
        ?id wdt:P31 ?instance .
        ?instance wdt:P465 ?hexcolor .
        ?instance p:P1343 ?stmt .
        ?stmt pq:P1476 ?iconname .
      }}

      OPTIONAL {{ ?id rdfs:label ?labelLang FILTER (lang(?labelLang) = '{lang}') }}
      OPTIONAL {{ ?id schema:description ?descLang FILTER (lang(?descLang) = '{lang}') }}
      OPTIONAL {{ ?id rdfs:label ?svLabel FILTER (lang(?svLabel) = 'sv') }}
      OPTIONAL {{ ?id schema:description ?svdesc FILTER (lang(?svdesc) = 'sv') }}
      OPTIONAL {{ ?svwiki schema:about ?id; schema:isPartOf <https://sv.wikipedia.org/> }}
      OPTIONAL {{ ?id wdt:P18 ?img }}
      OPTIONAL {{ ?id wdt:P3749 ?p3749 }}
      OPTIONAL {{ ?id wdt:P856 ?website }}
      OPTIONAL {{ ?id wdt:P2013 ?facebook }}
      OPTIONAL {{ ?id wdt:P2003 ?instagram }}
      OPTIONAL {{ ?id wdt:P4173 ?location }}
	  OPTIONAL {{
 		?id wdt:P2789 ?satSection .
  		?satSection rdfs:label ?satSectionLabel FILTER (lang(?satSectionLabel) = 'sv') .
    	OPTIONAL {{
    	    ?satSection p:P856 ?websiteStmt .
    	    ?websiteStmt ps:P856 ?satWebsite .
            OPTIONAL {{ ?websiteStmt pq:P407 ?websiteLang }}
        }}
      }} 
      OPTIONAL {{ ?id wdt:P11702 ?extraImage }}
      {filter_clause}
    }}
    GROUP BY ?id ?geo ?hexcolor ?iconname
    """

    headers = {"Accept": "application/sparql-results+json"}
    r = requests.get(endpoint_url, params={'query': query}, headers=headers)
    r.raise_for_status()
    results = r.json()

    rows = []
    for item in results['results']['bindings']:
        label = item.get('labelLangVal', {}).get('value', '-')
        description = item.get('descLangVal', {}).get('value', '')
        sv_label = item.get('svLabelVal', {}).get('value', '')
        sv_description = item.get('svdescVal', {}).get('value', '')
        geo = item.get('geo', {}).get('value')
        if not geo:
            continue
        coords = geo.replace('Point(', '').replace(')', '')

        lon, lat = map(float, coords.split(' '))
        color = '#' + item.get('hexcolor', {}).get('value', '228b22')
        symbol = item.get('iconname', {}).get('value', 'landmark-JP')
        img = item.get('imgVal', {}).get('value', '')
        wiki_url = item.get('svwikiVal', {}).get('value', '')
        qid = item.get('id', {}).get('value', '').split('/')[-1]
        gmaps = item.get('p3749Val', {}).get('value', '')
        website = item.get('websiteVal', {}).get('value', '')
        facebook = item.get('facebookVal', {}).get('value', '')
        instagram = item.get('instagramVal', {}).get('value', '')
        location = item.get('locationVal', {}).get('value', '')
        extra_img = item.get('extraImageVal', {}).get('value', '')

        popup_content = f"<b>{label}</b><br>"
        # 2-1) Add Stockholm Archipelago Trail text with link (if any)
        if website:
            popup_content += f"<a href='{website}' target='_blank'>🌐 Stockholm Archipelago Trail</a><br>"
        # 2-2) Add main image
        if img:
            popup_content += f"<br><img src='{img}' width='250'><br>"
        # 2-3) Add static Facebook group
        popup_content += f"<a href='https://www.facebook.com/groups/2875020699552247' target='_blank'>📘 Join us on Facebook</a><br>"

        # 2-4) Add optional extra image (P11702)
        if extra_img:
            popup_content += f"<br><img src='{extra_img}' width='250'><br>"
        # 2-5) Add Instagram hashtag link
        popup_content += (
            "<a href='https://www.instagram.com/explore/search/keyword/?q=%23stockholmarchipelagotrail' "
            "target='_blank'>📸 #stockholmarchipelagotrail</a>"
        )

        popup_content = f"<b>{lang}: {label} - {description}</b><br>"
        if lang != 'sv' and (sv_label or sv_description):
            popup_content += f"<b>sv: {sv_label} - {sv_description}</b><br>"
        popup_content += "<div style='margin-top:8px;'><b>Links:</b><br>"
        popup_content += f"<a href='https://www.wikidata.org/wiki/{qid}' title='Wikidata link' target='_blank'>🔗 Wikidata</a><br>"
        if gmaps:
            popup_content += f"<a href='https://maps.google.com/?cid={gmaps}' title='View on Google Maps' target='_blank'>🗺️ Google Maps</a><br>"
        if wiki_url:
            popup_content += f"<a href='{wiki_url}' target='_blank' title='Wikipedia (sv)'>📘 Wikipedia (sv)</a><br>"
        if website:
            popup_content += f"🌐 <a href='{website}' target='_blank' title='Website'>{website}</a><br>"
        if facebook:
            popup_content += f"📘 <a href='https://facebook.com/{facebook}' target='_blank' title='Facebook'>{facebook}</a><br>"
        if instagram:
            popup_content += f"📸 <a href='https://instagram.com/{instagram}' target='_blank' title='Instagram'>{instagram}</a>"
            if location:
                popup_content += f" — <a href='https://www.instagram.com/explore/locations/{location}' target='_blank' title='Instagram Location'>{location}</a>"
            popup_content += "<br>"
        if img:
            popup_content += f"<br><div style='margin-top:8px;'><b>Image:</b><br><img src='{img}' width='200' title='{label}' alt='{label}'></div>"

        #rows.append({'label': popup_content, 'lat': lat, 'lon': lon, 'color': color, 'symbol': symbol})
        rows.append({
            'label': popup_content,
            'lat': lat,
            'lon': lon,
            'color': color,
            'symbol': symbol,
        })

    df = pd.DataFrame(rows)

    #m = folium.Map(location=[59.3, 18.5], zoom_start=9) 
    m = folium.Map(location=[59.3, 18.1], zoom_start=10, width='100%', height='800px')
    # Add OSM SAT trail segments (GeoJSON)
    try:
        trail_geojson = fetch_osm_trail_geojson()
        folium.GeoJson(
            trail_geojson,
            name="SAT Trail",
            style_function=lambda x: {
                'color': 'red',
                'weight': 4,
                'opacity': 0.8
            },
            tooltip=folium.GeoJsonTooltip(
                fields=['name'],
                aliases=['Trail'],
                sticky=True
            )
    ).add_to(m)
    except Exception as e:
        print(f"Failed to load OSM trail: {e}")

    for item in rows:
        # If LINESTRING is available, draw trail
        if 'line' in item and item['line'].startswith("LINESTRING"):
            coords_str = item['line'].replace("LINESTRING(", "").replace(")", "")
            try:
                coords = [(float(lat), float(lon)) for lon, lat in [pair.strip().split() for pair in coords_str.split(",")]]
                folium.PolyLine(coords, color='blue', weight=4, opacity=0.7).add_to(m)
            except Exception as e:
                print("Failed to parse LINESTRING:", e)


    for _, row in df.iterrows():
        folium.CircleMarker(
            location=(row['lat'], row['lon']),
            radius=6,
            color=row['color'],
            fill=True,
            fill_color=row['color'],
            popup=row['label'],
            tooltip=row['label']
        ).add_to(m)





In [43]:
m 
# Interactive update
widgets.interact(fetch_and_show_map, lang=language_selector, instance=instance_selector)
m

interactive(children=(Dropdown(description='Language:', index=37, options=(('Arabic - ar', 'ar'), ('Chinese - …